In [5]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# Import necessary modules
import requests
import json

# Configuration for LLM API
LITELLM_HOST = "llmproxy-gcp.go-yubi.in"
API_URL = f"https://{LITELLM_HOST}/v1/chat/completions"

class ChatPrompt:
    def __init__(self):
        self.system_prompt = ""
        self.user_prompt = ""
    
    def set_system_prompt(self, prompt: str):
        self.system_prompt = prompt
        
    def set_user_prompt(self, prompt: str):
        self.user_prompt = prompt
        
    def get_messages(self):
        messages = []
        if self.system_prompt:
            messages.append({"role": "system", "content": self.system_prompt})
        messages.append({"role": "user", "content": self.user_prompt})
        return messages

def call_llm_api(messages, model="vertex-llama-api"):
    try:
        headers = {
            "Content-Type": "application/json",
        }
        data = {
            "model": model,
            "messages": messages
        }

        print(f"\nCalling LLM api with data: {json.dumps(data, indent=2)}")
        
        # Add timeout to prevent hanging
        response = requests.post(API_URL, headers=headers, json=data, timeout=30)
        
        print(f"\nResponse status code: {response.status_code}")
        print(f"Response headers: {dict(response.headers)}")
        
        response.raise_for_status()
        return response.json()
    except requests.exceptions.Timeout:
        print("Error: Request timed out. The API might be taking too long to respond.")
        return None
    except requests.exceptions.ConnectionError:
        print(f"Error: Could not connect to API at {API_URL}. Please check if the API is running.")
        return None
    except requests.exceptions.RequestException as e:
        print(f"HTTP Error: {str(e)}")
        print(f"Response text: {e.response.text if hasattr(e, 'response') and e.response else 'No response received'}")
        return None
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return None

# Example usage
def test_models():
    # Initialize prompt object
    chat = ChatPrompt()

    # Set system prompt
    system_prompt = "You are a helpful math tutor. Guide the user through the solution step by step."
    chat.set_system_prompt(system_prompt)

    # Set user prompt
    user_prompt = input("Enter your math question: ")
    chat.set_user_prompt(user_prompt)

    # Get messages
    messages = chat.get_messages()

    # Call API
    response = call_llm_api(messages)

    if response:
        print("\nAI Response:")
        print(response['choices'][0]['message']['content'])
    else:
        print("\nFailed to get response from AI")

# Run the function
test_models()

Enter your math question:  3+5



Calling LLM api with data: {
  "model": "vertex-llama-api",
  "messages": [
    {
      "role": "system",
      "content": "You are a helpful math tutor. Guide the user through the solution step by step."
    },
    {
      "role": "user",
      "content": "3+5"
    }
  ]
}
Error: Could not connect to API at https://llmproxy-gcp.go-yubi.in/v1/chat/completions. Please check if the API is running.

Failed to get response from AI


In [3]:
def test_api_connection():
    try:
        test_data = {
            "model": "vertex-llama-api",
            "messages": [
                {"role": "user", "content": "Hello"}
            ]
        }
        print(f"Testing API connection with test data: {json.dumps(test_data, indent=2)}")
        
        response = requests.post(API_URL, headers={"Content-Type": "application/json"}, json=test_data, timeout=10)
        print(f"Test response status: {response.status_code}")
        print(f"Test response body: {response.text}")
        
        return response.status_code == 200
    except Exception as e:
        print(f"API test failed: {str(e)}")
        return False

# Run test before main function
if not test_api_connection():
    print("API connection test failed. Please check:")
    print(f"1. Is the Litellm service running at {LITELLM_HOST}?")
    print("2. Are you connected to the correct network?")
    print("3. Is the API URL correct?")
else:
    print("API connection test successful!")

Testing API connection with test data: {
  "model": "vertex-llama-api",
  "messages": [
    {
      "role": "user",
      "content": "Hello"
    }
  ]
}
API test failed: HTTPSConnectionPool(host='llmproxy-gcp.go-yubi.in', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)')))
API connection test failed. Please check:
1. Is the Litellm service running at llmproxy-gcp.go-yubi.in?
2. Are you connected to the correct network?
3. Is the API URL correct?
